<a href="https://colab.research.google.com/github/oliveiraprg/text-summarization-with-python/blob/main/algoritimo_de_luhn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sumarização de textos com o Algoritimo de Luhn 



*  [ https://courses.ischool.berkeley.edu/i256/f06/papers/luhn58.pdf](https://courses.ischool.berkeley.edu/i256/f06/papers/luhn58.pdf)




## Preparação do texto de exemplo

In [1]:
import re
import nltk
import numpy
import string
import heapq

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
texto_original = """A inteligência artificial é a inteligência similar à humana máquinas.
                    Definem como o estudo de agente artificial com inteligência.
                    Ciência e engenharia de produzir máquinas com inteligência.
                    Resolver problemas e possuir inteligência. 
                    Relacionada ao comportamento inteligente. 
                    Construção de máquinas para raciocinar. 
                    Aprender com os erros e acertos. 
                    Inteligência artificial é raciocinar nas situações do cotidiano."""

In [5]:
texto_original = re.sub(r'\s+', ' ', texto_original)

In [6]:
stop_words = nltk.corpus.stopwords.words('portuguese')

In [25]:
def pre_processamento(texto):
  texto_formatado = texto.lower()
  tokens = []
  for token in nltk.word_tokenize(texto_formatado):
    tokens.append(token)
  tokens = [palavra for palavra in tokens if palavra not in stop_words and palavra not in string.punctuation]
  texto_formatado = ' '.join([str(elemento) for elemento in tokens if not elemento.isdigit()])
  return texto_formatado

## Função para calcular a nota das sentenças

In [33]:
def calcula_nota_sentenca(sentencas, palavras_importantes, distancia):
  notas = []
  indice_sentenca = 0

  for sentenca in [nltk.word_tokenize(sentenca.lower()) for sentenca in sentencas]:
    indice_palavra = []
    for palavra in palavras_importantes:
      try:
        indice_palavra.append(sentenca.index(palavra))
      except ValueError:
        pass
    indice_palavra.sort()
    if len(indice_palavra) == 0:continue  
    lista_grupos = []
    grupo_palavras = [indice_palavra[0]]
    i = 1
    while i < len(indice_palavra):
      if indice_palavra[i] - indice_palavra[i-1] < distancia:
        grupo_palavras.append(indice_palavra[i])
      else:
        lista_grupos.append(grupo_palavras[:])
        grupo_palavras = [indice_palavra[i]]
      i += 1
    lista_grupos.append(grupo_palavras)
    nota_maxima_grupo = 0
    for grupo in lista_grupos:
      palavras_importantes_no_grupo = len(grupo)
      total_palavras_no_grupo = grupo[-1] - grupo[0] + 1
      nota = 1.0 * palavras_importantes_no_grupo**2 / total_palavras_no_grupo
      if nota > nota_maxima_grupo:
        nota_maxima_grupo = nota
    notas.append((nota_maxima_grupo, indice_sentenca))
    indice_sentenca += 1
  return notas


## Função para sumarizar os textos

In [53]:
def sumarizar(texto, distancia):
  sentencas_originais = [sentenca for sentenca in nltk.sent_tokenize(texto)]
  sentencas_formatadas = [pre_processamento(sentenca_original) for sentenca_original in sentencas_originais]
  palavras = [palavra for sentenca in sentencas_formatadas for palavra in nltk.word_tokenize(sentenca)]
  top_n_palavras = int(len(palavras) / 4) 
  frequencia = nltk.FreqDist(palavras)
  quantidade_sentencas = int(len(sentencas_originais) / 2)
  top_n_palavras = [palavra[0] for palavra in frequencia.most_common(top_n_palavras)]
  calcula_nota_sentenca(sentencas_formatadas, top_n_palavras, distancia)
  notas_sentencas = calcula_nota_sentenca(sentencas_formatadas, top_n_palavras, distancia)
  melhores_sentencas = heapq.nlargest(quantidade_sentencas, notas_sentencas)
  melhores_sentencas = [sentencas_originais[i] for (nota, i) in melhores_sentencas]
  return sentencas_originais, melhores_sentencas, notas_sentencas

In [54]:
sentencas_originais, melhores_sentencas, notas_sentencas = sumarizar(texto_original, 5)

In [55]:
sentencas_originais, melhores_sentencas, notas_sentencas

(['A inteligência artificial é a inteligência similar à humana máquinas.',
  'Definem como o estudo de agente artificial com inteligência.',
  'Ciência e engenharia de produzir máquinas com inteligência.',
  'Resolver problemas e possuir inteligência.',
  'Relacionada ao comportamento inteligente.',
  'Construção de máquinas para raciocinar.',
  'Aprender com os erros e acertos.',
  'Inteligência artificial é raciocinar nas situações do cotidiano.'],
 ['A inteligência artificial é a inteligência similar à humana máquinas.',
  'Definem como o estudo de agente artificial com inteligência.',
  'Construção de máquinas para raciocinar.',
  'Relacionada ao comportamento inteligente.'],
 [(4.166666666666667, 0), (3.2, 1), (2.0, 2), (1.0, 3), (2.0, 4), (3.0, 5)])

## Visualização do resumo

In [56]:
def visualiza_resumo(titulo, lista_sentencas, melhores_sentencas):
  from IPython.core.display import HTML
  texto = ''
  resumo = ''
  display(HTML(f'<h1>{titulo}</h1>'))
  display(HTML(f'<h2>Texto com resumo destacado - {titulo}</h2>'))
  for sentenca in lista_sentencas:
    if sentenca in melhores_sentencas:
      texto += str(sentenca).replace(sentenca, f'<mark>{sentenca} </mark>')
      resumo += str(sentenca).replace(sentenca, f'{sentenca} ')
    else:
      texto += f'{sentenca} '
  display(HTML(f'{texto}'))
  display(HTML(f'<h2>Resumo de - {titulo}</h2>'))
  display(HTML(f'{resumo}'))

In [57]:
visualiza_resumo('Inteligência Artificial', sentencas_originais, melhores_sentencas)

## Extração de texto da internet

In [58]:
!pip install goose3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [59]:
from goose3 import Goose

In [60]:
g = Goose()
url = 'https://iaexpert.academy/2020/11/09/ia-preve-resultado-das-eleicoes-americanas/'
artigo = g.extract(url)

In [61]:
artigo.cleaned_text

'Nas eleições presidenciais americanas de 2016, a maioria das predições apontavam para a vitória de Hillary Clinton. Entretanto, a história nos mostrou o resultado oposto, e Donald Trump foi o presidente nos últimos 4 anos. Desta vez, os estatísticos reexaminaram seus modelos, para aumentar o grau de confiabilidade nos seus resultados. Nesta tentativa de otimização das predições, a inteligência artificial certamente não ficou de fora.\n\nO modelo desenvolvido pelo Dr. Hernan Makse, físico estatístico da Universidade da Cidade de Nova York, baseou suas predições em uma rede neural treinada para processar os sentimentos expressos nas redes sociais. O algoritmo fez a análise de cerca de 1 bilhão de tweets para chegar a uma estimativa dos resultados do pleito. No dia da eleição, 3 de novembro, o modelo estava indicando a vitória de Joe Biden.\n\nO Dr. Makse disse que seu trabalho começou já na eleição de 2016, e foi testado novamente nas eleições na Argentina ano passado. Desta vez, o mode

In [62]:
sentencas_originais, melhores_sentencas, notas_sentencas = sumarizar(artigo.cleaned_text, 5)

In [63]:
sentencas_originais, melhores_sentencas, notas_sentencas

(['Nas eleições presidenciais americanas de 2016, a maioria das predições apontavam para a vitória de Hillary Clinton.',
  'Entretanto, a história nos mostrou o resultado oposto, e Donald Trump foi o presidente nos últimos 4 anos.',
  'Desta vez, os estatísticos reexaminaram seus modelos, para aumentar o grau de confiabilidade nos seus resultados.',
  'Nesta tentativa de otimização das predições, a inteligência artificial certamente não ficou de fora.',
  'O modelo desenvolvido pelo Dr. Hernan Makse, físico estatístico da Universidade da Cidade de Nova York, baseou suas predições em uma rede neural treinada para processar os sentimentos expressos nas redes sociais.',
  'O algoritmo fez a análise de cerca de 1 bilhão de tweets para chegar a uma estimativa dos resultados do pleito.',
  'No dia da eleição, 3 de novembro, o modelo estava indicando a vitória de Joe Biden.',
  'O Dr. Makse disse que seu trabalho começou já na eleição de 2016, e foi testado novamente nas eleições na Argentina

In [64]:
visualiza_resumo(artigo.title, sentencas_originais, melhores_sentencas)